<a href="https://colab.research.google.com/github/AnoVando/MSIS/blob/master/MSIS521_IA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') # tokenizer
nltk.download('wordnet') # lemmatizer
nltk.download('stopwords') # used to handle words like a, an, the
nltk.download('averaged_perceptron_tagger') # Part of Speech

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)

from sklearn.decomposition import LatentDirichletAllocation



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
url = 'https://github.com/AnoVando/MSIS/raw/master/IA3.csv'
data = pd.read_csv(url, header='infer')
reviews = data['review'].tolist()

In [31]:
print(data[:5])

   id                                             review       label
0   1  About the shop: There is a restaurant in Soi L...  restaurant
1   2  About the shop: Through this store for about t...  restaurant
2   3  Roast Coffee &amp; Eatery is a restaurant loca...  restaurant
3   4  Eat from the children. The shop is opposite. P...  restaurant
4   5  The Ak 1 shop at another branch tastes the sam...  restaurant


Part 1. Topic Model
There are 1000 reviews for restaurants and films in a collection under the attached csv file. All of those
reviews are saved as text files. In this assignment, you are required to investigate the topics of those
reviews. In particular, please follow the steps listed below:
1. Transform those reviews into a term‐document matrix, lemmatize all the words, remove the
stop‐words and punctuations, set the minimal document frequency for each term to be 5 and
include 2‐gram.
2. Use the LDA model to extract the topics of each document. In particular, we assume there are 6
topics.
3. Report the topic distribution and the top‐2 topics of the first 10 restaurant reviews (id = [1:10])
and the first 10 movie reviews (id = [501:510]).
4. Find the top‐5 terms (terms with the top‐5 highest weights) for each of the 6 topics. Based on
those terms, describe what those topics are about.
5. Based on finding in 3 and 4, describe what review 1 [ID=1] and review 501 [ID=501] are about?
Please submit 1 file:
A word file includes python code with your comment #, and one screenshot on your Jupyter
Notebook showing that your code has run through successfully for each of the first four steps (4
screenshots in total). Also, report your answers to question 3, 4, and 5 at the end of the word
file.

In [0]:
# Iterate through the list and tokenize each comment
tokens = []
for row in data.review:
  review = nltk.word_tokenize(row)
  tokens.append(review)

In [0]:
# Perform lemmatization on all of the words in the comments
lems = []
lemmatizer = nltk.stem.WordNetLemmatizer()
for token in tokens:
  lemmatized_review = [lemmatizer.lemmatize(x) for x in token if x.isalpha()]
  lems.append(lemmatized_review)

In [0]:
# Remove Stopwords
stops = []
for lem in lems:
  stop_words_removed = [x for x in lem if not x in stopwords.words('english')]
  stops.append(stop_words_removed)

In [39]:
# Generate TF-IDF Vectors
reviews = [" ".join(x) for x in stops]

tfidf = TfidfVectorizer(ngram_range=(2, 2), min_df=5) # 2-grams and min. document frequency of 5
tfidf.fit(reviews)
vector = tfidf.transform(reviews)
vector = vector.toarray()
print(tfidf.vocabulary_)
print(vector)
print(vector.shape)

{'about shop': 0, 'shop there': 1838, 'hotel the': 848, 'restaurant decorated': 1718, 'average price': 101, 'price baht': 1476, 'soup sweet': 1885, 'sweet taste': 1965, 'cream cheese': 291, 'eat together': 414, 'topic lt': 2286, 'atmosphere nice': 91, 'lt food': 1142, 'food taste': 611, 'taste gt': 1980, 'gt lt': 739, 'lt service': 1143, 'service gt': 1814, 'gt good': 738, 'good service': 723, 'service good': 1813, 'good food': 699, 'lt value': 1144, 'price quite': 1481, 'quite expensive': 1499, 'the atmosphere': 2004, 'atmosphere good': 90, 'good delicious': 695, 'delicious food': 332, 'note this': 1306, 'this review': 2217, 'we like': 2455, 'like if': 1050, 'if please': 871, 'please forgive': 1458, 'forgive think': 622, 'think one': 2188, 'one comment': 1326, 'comment to': 269, 'to win': 2277, 'win competition': 2480, 'competition good': 276, 'good restaurant': 721, 'restaurant thank': 1726, 'three year': 2237, 'this first': 2203, 'first time': 592, 'time try': 2265, 'try the': 2297,

In [0]:
# Employ LDA Model
lda = LatentDirichletAllocation(n_components=6).fit(vector)

